In [1]:
%load_ext autoreload
%autoreload 2

from primitives.primitives import *
import matplotlib.pyplot as plt
from numpy.linalg import eig
import qutip as qt

In [2]:
def commute(a, b):
    a = np.array(a, dtype=np.complex128)
    b = np.array(b, dtype=np.complex128)

    c = a@b
    d = b@a
    if np.allclose(c-d, np.zeros(c.shape)):
        print("Commute")
    elif np.allclose(c+d, np.zeros(c.shape)):
        print("Anti-commute")
    else:
        print("Null")
        
def is_unitary(m):
    return np.allclose(np.eye(m.shape[0]), m.dot(m.conj().T))

In [4]:
a = TensorProduct(sy_gamma_5*sy_gamma_2, sy_gamma_5)
b = TensorProduct(sy_gamma_5*sy_gamma_1, sy_gamma_1)

a = sy_gamma_1*sy_gamma_2
b = sy_gamma_2

commute(a,b)

Anti-commute


In [5]:
from scipy.linalg import eig
def find_unitary_transform(A, B):
    """
    Find the unitary matrix U such that U A U^dagger = B.

    Parameters:
    A (ndarray): Input matrix A.
    B (ndarray): Input matrix B.

    Returns:
    U (ndarray or None): Unitary matrix U if it exists, otherwise None.
    """
    A = np.array(A, complex)
    B = np.array(B, complex)
    
    # Get the eigenvalues and eigenvectors of A
    eigvals_A, eigvecs_A = eig(A)
    eigvals_B, eigvecs_B = eig(B)
    
    # Sort the eigenvalues and corresponding eigenvectors for comparison
    idx_A = np.argsort(eigvals_A)
    idx_B = np.argsort(eigvals_B)
    
    eigvals_A_sorted = eigvals_A[idx_A]
    eigvals_B_sorted = eigvals_B[idx_B]
    
    eigvecs_A_sorted = eigvecs_A[:, idx_A]
    eigvecs_B_sorted = eigvecs_B[:, idx_B]

    # Check if the sorted eigenvalues match
    if not np.allclose(eigvals_A_sorted, eigvals_B_sorted):
        print("not all eigenvalues match")
        print(eigvals_A_sorted)
        print(eigvals_B_sorted)
        return None

    eigvecs_A_sorted = np.linalg.qr(eigvecs_A_sorted)[0]
    eigvecs_B_sorted = np.linalg.qr(eigvecs_B_sorted)[0]

    # Construct the unitary matrix U
    U = eigvecs_B_sorted @ eigvecs_A_sorted.conj().T
    
    # Check if U is unitary
    if np.allclose(U @ U.conj().T, np.eye(U.shape[0])):
        return U
    else:
        print("not unitary")
        return U

In [ ]:
def to_latex(mat):
    text = r"\begin{pmatrix}"
    text += "\n"
    for row in mat:
        for idx, el in enumerate(row):
            if np.imag(el) != 0.0:
                raise Exception
            el = np.real(el)
            if el%1 == 0:
                text += str(int(el))
            elif np.abs(el) == 0.7071068:
                if el < 0:
                    text += "-"
                text += r"\sqrt{0.5}"
            else:
                text += str(el)
            
            if idx + 1 < len(row):
                text += " & "
        text += r"\\"
        text += "\n"
        

    text += r"\end{pmatrix}"
    return text

### Spinless Unitary

In [63]:
# horizontal

a = 1j * TensorProduct(sy_gamma_1, sy_gamma_2) * ( TensorProduct(sy_gamma_5, sy_id) - TensorProduct(sy_id, sy_gamma_5))
b = TensorProduct(sy_gamma_5, sy_id) + TensorProduct(sy_id, sy_gamma_5)

U = find_unitary_transform(a, b)

np.allclose(U @ U.conj().T, np.eye(U.shape[0]), rtol=1e-10)

True

In [75]:
na = np.array(a, dtype=np.complex128)
nprod = np.array(U.conj().T @ b @ U, dtype=np.complex128)

np.allclose(na, nprod, rtol=1e-6)

True

In [9]:
new_U = np.round(U, 7)
print(to_latex(new_U))

\begin{pmatrix}
0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
-1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0\\
0 & 0 & 0 & 0 & 0 

In [89]:
# vertical

a = 1j * TensorProduct(sy_gamma_3, sy_gamma_4) * ( TensorProduct(sy_gamma_5, sy_id) - TensorProduct(sy_id, sy_gamma_5))
b = TensorProduct(sy_gamma_5, sy_id) + TensorProduct(sy_id, sy_gamma_5)

U = find_unitary_transform(a, b)

np.allclose(U @ U.conj().T, np.eye(U.shape[0]), rtol=1e-10)

True

In [90]:
na = np.array(a, dtype=np.complex128)
nprod = np.array(U.conj().T @ b @ U, dtype=np.complex128)

np.allclose(na, nprod, rtol=1e-6)

True

In [91]:
new_U = np.round(U, 7)
print(to_latex(new_U))

\begin{pmatrix}
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0\\
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0\\
0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & -\sqrt{0.5} & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & \sqrt{0.5} & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\
0 & -\sqrt{0.5} & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 

### Spinful (2)

In [78]:
# spinful 2 horizontal

#a = 1j*TensorProduct(sy_gamma_5, sy_id, sy_gamma_5) * ( TensorProduct(sy_gamma_1, sy_gamma_2, sy_id) - TensorProduct(sy_gamma_2, sy_gamma_1, sy_id))
a = 1j*TensorProduct(sy_gamma_5, sy_id) * ( TensorProduct(sy_gamma_1, sy_gamma_2) - TensorProduct(sy_gamma_2, sy_gamma_1))
b = TensorProduct(sy_gamma_5, sy_id) + TensorProduct(sy_id, sy_gamma_5)

U = find_unitary_transform(a, b)

np.allclose(U @ U.conj().T, np.eye(U.shape[0]), rtol=1e-6)

True

In [79]:
na = np.array(a, dtype=np.complex128)
nprod = np.array(U.conj().T @ b @ U, dtype=np.complex128)

np.allclose(na, nprod, rtol=1e-6)

True

In [81]:
new_U = np.round(na, 7)
print(to_latex(new_U))

\begin{pmatrix}
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -2 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -2 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 2 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 2 & 0 & 0\\
0 & 0 & -2 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & -2 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 2 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 2 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 

In [83]:
# spinful 2 vertical

a = TensorProduct(sy_gamma_5, sy_gamma_5) * ( TensorProduct(sy_gamma_1, sy_gamma_2) - TensorProduct(sy_gamma_2, sy_gamma_1))
b = TensorProduct(sy_gamma_5, sy_id) + TensorProduct(sy_id, sy_gamma_5)

U = find_unitary_transform(a, b)

np.allclose(U @ U.conj().T, np.eye(U.shape[0]), rtol=1e-10)

True

In [84]:
na = np.array(a, dtype=np.complex128)
nprod = np.array(U.conj().T @ b @ U, dtype=np.complex128)

np.allclose(na, nprod, rtol=1e-6)

True

In [88]:
new_U = 1j * np.round(na, 7)
print(to_latex(new_U))

\begin{pmatrix}
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 2 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -2 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -2 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 2 & 0 & 0\\
0 & 0 & -2 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 2 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 2 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & -2 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 

In [87]:
Matrix(new_U)

Matrix([
[0, 0,     0,      0, 0, 0,      0,     0,      0,     0, 0, 0,     0,      0, 0, 0],
[0, 0,     0,      0, 0, 0,      0,     0,      0,     0, 0, 0,     0,      0, 0, 0],
[0, 0,     0,      0, 0, 0,      0,     0, -2.0*I,     0, 0, 0,     0,      0, 0, 0],
[0, 0,     0,      0, 0, 0,      0,     0,      0, 2.0*I, 0, 0,     0,      0, 0, 0],
[0, 0,     0,      0, 0, 0,      0,     0,      0,     0, 0, 0,     0,      0, 0, 0],
[0, 0,     0,      0, 0, 0,      0,     0,      0,     0, 0, 0,     0,      0, 0, 0],
[0, 0,     0,      0, 0, 0,      0,     0,      0,     0, 0, 0, 2.0*I,      0, 0, 0],
[0, 0,     0,      0, 0, 0,      0,     0,      0,     0, 0, 0,     0, -2.0*I, 0, 0],
[0, 0, 2.0*I,      0, 0, 0,      0,     0,      0,     0, 0, 0,     0,      0, 0, 0],
[0, 0,     0, -2.0*I, 0, 0,      0,     0,      0,     0, 0, 0,     0,      0, 0, 0],
[0, 0,     0,      0, 0, 0,      0,     0,      0,     0, 0, 0,     0,      0, 0, 0],
[0, 0,     0,      0, 0, 0,      0,     0,   